In [5]:
from bs4 import BeautifulSoup
import urllib
import requests

#state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = ["Michigan"]
for state in state_names:
  url = "https://www.indeed.com/jobs?q=data+scientist&l=" + state
  for page_number in range (0,1):
    url = url + "&start=" + str(10*page_number)
    #print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    
    for job in soup.find_all("a", class_ = "jobtitle turnstileLink "): 
      job_url  = "https://www.indeed.com" + job['href'] 
      print(job_url)
      try:
        job_soup = BeautifulSoup(urllib.request.urlopen(job_url)) 
      except:
        pass
      #print(job_soup.prettify()[:50])

      for title in job_soup.find_all("h3", class_="jobsearch-JobInfoHeader-title"):
        print(title.get_text())
      for company in job_soup.find_all("div", class_ = "icl-u-lg-mr--sm icl-u-xs-mr--xs", limit=1):
        print(company.get_text())
      for salary in job_soup.find_all("span", class_ = "icl-u-xs-mr--xs"):
        print(salary.get_text())

https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BTYkY06FZEdAAtNWO-eDAfNklmfZymsMF6eFRONl7rAMN5x_2sHrqXfWPo9rHDxSN5QVY9jItzMNiKGmQ1CNqv-eLRP-g4viOJBX-8A-i2wkPm5JofkG5TBT-b0Y1Xotu4g-JPbJ9Z6wy_yvbfZfu0JCwNiS6ybGOSECGttf63brhI0vAbfKM_t2gAPzcFern387VkspCaDlzL23QhNQclemcA8eAcUbfzneD9U8wlTwoJFriGVQix4x6bM6BzHu41ZtqvR4EswFZI2Cdidl500dh8yP2ZvrXlFAJUW91U8FLOAwFQCmXIYOt1f2WgB9FUf-8LLON3bIAaooP0vANDcAmc12VSz_Azq_q0OdmhNjXOQFxuXODsWAsSVnGJQTN6HPUYMz3S26ZjOykxnChPEqCy5IcwuimK1LI8Xk7ypio0q_LagtSwRMJvGlwf0-i-ho1QFvF4iw==&p=0&fvj=1&vjs=3
Data Scientist
DISYS
$65 an hour
https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0ACPwgM8vN-agjfeQIp8j7bA6rWcStjIJMvSUoZk9GVGafyfcR9vdi_Ifoij-dP1hPEUoAXKt7Tq16GSM4YzcEBkNv904qiOYJ_xCSguH_q1cSpvX3bURrBFxClywVTxU351nrykd9rXZsIF3JmgFiWpRrsSu7sWZoAi-b00azOSPqepmZNOi9bj6W6o1XvPSHyx7znxmG1V6hJlZB_sLHPbXDUOaZ021JWsGqFuxy42-0xSgZL5TEXaLgtDHpWQhK--XadSFtMnZMvH8Y8jzoynaarhmU48pn20tOaUm2tXleqopJ24iMIcI4hWqfOLaQjz06PlesalymwLYe13kgzuiA_WwsbO1YC4ypMdYTaA_u7nFMz5WXMbTdbcan85Z